In [2]:
import gpxpy
import pandas as pd
import plotly.graph_objects as go

def parse_gpx(file_path):
    # Parse the GPX file
    with open(file_path, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    data = {
        'latitude': [],
        'longitude': [],
        'elevation': [],
        'distance': []
    }

    # Extract data
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                data['latitude'].append(point.latitude)
                data['longitude'].append(point.longitude)
                data['elevation'].append(point.elevation)
                if len(data['distance']) == 0:
                    data['distance'].append(0)
                else:
                    prev_point = segment.points[len(data['distance']) - 1]
                    dist = point.distance_3d(prev_point)
                    data['distance'].append(data['distance'][-1] + dist)

    return pd.DataFrame(data)

def create_3d_plot(df):
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
        x=df['longitude'],
        y=df['latitude'],
        z=df['elevation'],
        mode='lines',
        line=dict(
            color=df['elevation'],
            colorscale='Viridis',
            width=2
        )
    ))

    fig.update_layout(
        scene=dict(
            xaxis_title='Longitude',
            yaxis_title='Latitude',
            zaxis_title='Elevation',
            zaxis=dict(nticks=10, range=[0, df['elevation'].max()])
        ),
        title='3D Course Profile',
        margin=dict(l=0, r=0, b=0, t=50)
    )

    fig.show()

# Example usage
gpx_file_path = 'gpx_race/Ecotrail.gpx'  # Replace with your GPX file path
df = parse_gpx(gpx_file_path)
create_3d_plot(df)
